In [1]:
import pandas as pd
import numpy as np

In [2]:
#load dataframe for NAICS code to description
naics17 = pd.DataFrame(pd.read_csv('../data/naics17.txt', sep=","))
naics12 = pd.DataFrame(pd.read_csv('../data/naics12.txt', sep=","))
naics08 = pd.DataFrame(pd.read_csv('../data/naics08.csv'))
naics03 = pd.DataFrame(pd.read_csv('../data/naics03.csv'))
naics98 = pd.DataFrame(pd.read_csv('../data/naics98.csv'))

#clean dataframes
naics12['DESCRIPTION'] = list(map(lambda x: x[:-1], naics12['DESCRIPTION']))

#turn dataframe into dictionary
naics17 = naics17.set_index("NAICS").to_dict()['DESCRIPTION']
naics12 = naics12.set_index("NAICS").to_dict()['DESCRIPTION']
naics08 = naics08.set_index("NAICS").to_dict()['DESCRIPTION']
naics03 = naics03.set_index("NAICS").to_dict()['DESCRIPTION']
naics98 = naics98.set_index("NAICS").to_dict()['DESCRIPTION']

#import zip to FIPS, zip to County, zip to State
zipFIPS = pd.DataFrame(pd.read_csv('../data/zip_FIPS.csv'), dtype=str)[["ZIP", "STCOUNTYFP"]]
zipCOUNTY = pd.DataFrame(pd.read_csv('../data/zip_FIPS.csv', dtype = str))[["ZIP", "COUNTYNAME"]]
zipSTATE = pd.DataFrame(pd.read_csv('../data/zip_FIPS.csv', dtype = str))[["ZIP", "STATE"]]

#fixing problematic imports
zipFIPS['STCOUNTYFP'] = list (map(lambda x: "0" + x if len(x) == 4 else x, zipFIPS['STCOUNTYFP']))

#turn dataframe into dictionary
zipFIPS = zipFIPS.set_index("ZIP").to_dict()["STCOUNTYFP"]
zipCOUNTY = zipCOUNTY.set_index("ZIP").to_dict()["COUNTYNAME"]
zipSTATE = zipSTATE.set_index("ZIP").to_dict()["STATE"]

#Selected industries
indices = ['------', '44----', '54----', '62----', '81----', '23----', '72----', '52----', '42----', '56----', '53----', '31----']

In [3]:
def cleanDf(file, naics):
    #importing small businesses dataset, casting to int
    sb = pd.read_csv(file, sep=",", dtype = str)
    #find the yearo f the dataset
    year = int("20" + file[11:13])
    #year correction from 2098, 2099 to 1998, 1999
    if (year > 2021):
        year = year - 100
    
    #make all columns lowercase to standardize
    sb.columns = [s.lower() for s in sb.columns]

    
    #data from 1998 has incorrectly named zip column
    if (year == 1998):
        col = list(sb.columns)
        col[0] = "zip"
        sb.columns = col
        
    #standardize column names
    if ("n1_4" not in sb.columns):
        sb = sb.rename(columns = {"n<5":"n1_4"})
    
    #initial dimensions of the dataset
    print(file)
    print(sb.shape)

    #filer for desired naics label - remove for more detail
    #sb = sb[sb['naics'].isin(["------"])]
    
    sb = sb.loc[sb['naics'].isin(indices)]


    #Add FIPS for zip, remove invalid ZIP
    sb = sb[sb['zip'].isin(zipFIPS.keys())]    
    sb['FIPS'] = list(map(lambda x: zipFIPS[x], sb['zip']))
    
    #adding and filtering  state based off zip classification
    sb['state'] = list(map(lambda x: zipSTATE[x], sb['zip']))
    #sb = sb[sb['state'].isin(states)]
    
    #add counties 
    sb['county'] = list(map(lambda x: zipCOUNTY[x], sb['zip']))

    #adding corresponding industry descriptions for each naics code
    if (naics == 17):
        sb['naics_desc'] = list(map(lambda x: naics17[x], sb['naics']))
    elif (naics == 12):
        sb['naics_desc'] = list(map(lambda x: naics12[x], sb['naics']))
    elif (naics == 8):
        sb['naics_desc'] = list(map(lambda x: naics08[x], sb['naics']))
    elif (naics == 3):
        sb['naics_desc'] = list(map(lambda x: naics03[x], sb['naics']))
    elif (naics == 98):
        sb['naics_desc'] = list(map(lambda x: naics98[x], sb['naics']))
        
    #add year column
    sb['year'] = year
    
    #final dataset dim
    print(sb.shape)
    
    return sb

In [4]:
sb18 = cleanDf('../data/zbp18detail.txt', 17)
sb17 = cleanDf('../data/zbp17detail.txt', 17)
sb16 = cleanDf('../data/zbp16detail.txt', 12)
sb15 = cleanDf('../data/zbp15detail.txt', 12)
sb14 = cleanDf('../data/zbp14detail.txt', 12)
sb13 = cleanDf('../data/zbp13detail.txt', 12)
sb12 = cleanDf('../data/zbp12detail.txt', 12)
sb11 = cleanDf('../data/zbp11detail.txt', 8)
sb10 = cleanDf('../data/zbp10detail.txt', 8)
sb09 = cleanDf('../data/zbp09detail.txt', 8)
sb08 = cleanDf('../data/zbp08detail.txt', 8)
sb07 = cleanDf('../data/zbp07detail.txt', 3)
sb06 = cleanDf('../data/zbp06detail.txt', 3)
sb05 = cleanDf('../data/zbp05detail.txt', 3)
sb04 = cleanDf('../data/zbp04detail.txt', 3)
sb03 = cleanDf('../data/zbp03detail.txt', 3)
sb02 = cleanDf('../data/zbp02detail.txt', 98)
sb01 = cleanDf('../data/zbp01detail.txt', 98)
sb00 = cleanDf('../data/zbp00detail.txt', 98)
sb99 = cleanDf('../data/zbp99detail.txt', 98)
sb98 = cleanDf('../data/zbp98detail.txt', 98)

../data/zbp18detail.txt
(2874446, 12)
(212934, 17)
../data/zbp17detail.txt
(2870579, 12)
(213199, 17)
../data/zbp16detail.txt
(8418283, 12)
(300564, 17)
../data/zbp15detail.txt
(8403660, 12)
(300896, 17)
../data/zbp14detail.txt
(8377070, 12)
(301033, 17)
../data/zbp13detail.txt
(8371007, 12)
(301285, 17)
../data/zbp12detail.txt
(8380408, 12)
(301864, 17)
../data/zbp11detail.txt
(3196767, 12)
(301931, 17)
../data/zbp10detail.txt
(3219077, 12)
(303238, 17)
../data/zbp09detail.txt
(3238421, 12)
(304193, 17)
../data/zbp08detail.txt
(3290137, 12)
(305857, 17)
../data/zbp07detail.txt
(3340643, 12)
(307061, 17)
../data/zbp06detail.txt
(3321529, 12)
(307065, 17)
../data/zbp05detail.txt
(3307690, 12)
(307128, 17)
../data/zbp04detail.txt
(3296175, 12)
(306520, 17)
../data/zbp03detail.txt
(3267445, 12)
(304723, 17)
../data/zbp02detail.txt
(3251827, 12)
(305494, 17)
../data/zbp01detail.txt
(3279975, 12)
(307657, 17)
../data/zbp00detail.txt
(3260522, 12)
(306482, 17)
../data/zbp99detail.txt
(322459

In [5]:
allYears = pd.concat([sb18, sb17, sb16, sb15, sb14, sb13, sb12, sb11, sb10,
                      sb09, sb08, sb07, sb06, sb05, sb04, sb03, sb02, sb01, sb00,
                      sb99, sb98], ignore_index=True)
allYears.head()

,zip,naics,est,n1_4,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,FIPS,state,county,naics_desc,year
0,10001,------,7248,4052,1222,820,647,254,156,62,24,11,36061,NY,New York County,Total for all sectors,2018
1,10001,23----,262,90,46,54,45,14,8,4,N,N,36061,NY,New York County,Construction,2018
2,10001,31----,188,107,31,36,11,N,N,N,N,N,36061,NY,New York County,Manufacturing,2018
3,10001,42----,1079,662,198,109,69,23,15,3,N,N,36061,NY,New York County,Wholesale Trade,2018
4,10001,44----,654,387,127,62,38,18,8,11,N,N,36061,NY,New York County,Retail Trade,2018


In [6]:
allYears.shape

(6207190, 17)

In [7]:
allYears["est"] = pd.to_numeric(allYears["est"], errors = "coerce")
allYears["n1_4"] = pd.to_numeric(allYears["n1_4"], errors = "coerce")
allYears["n5_9"] = pd.to_numeric(allYears["n5_9"], errors = "coerce")
allYears["n10_19"] = pd.to_numeric(allYears["n10_19"], errors = "coerce")
allYears["n20_49"] = pd.to_numeric(allYears["n20_49"], errors = "coerce")
allYears["n50_99"] = pd.to_numeric(allYears["n50_99"], errors = "coerce")
allYears["n100_249"] = pd.to_numeric(allYears["n100_249"], errors = "coerce")
allYears["n250_499"] = pd.to_numeric(allYears["n250_499"], errors = "coerce")
allYears["n500_999"] = pd.to_numeric(allYears["n500_999"], errors = "coerce")
allYears["n1000"] = pd.to_numeric(allYears["n1000"], errors = "coerce")

In [8]:
allYears.head()

,zip,naics,est,n1_4,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,FIPS,state,county,naics_desc,year
0,10001,------,7248,4052.0,1222.0,820.0,647.0,254.0,156.0,62.0,24.0,11.0,36061,NY,New York County,Total for all sectors,2018
1,10001,23----,262,90.0,46.0,54.0,45.0,14.0,8.0,4.0,NaN,NaN,36061,NY,New York County,Construction,2018
2,10001,31----,188,107.0,31.0,36.0,11.0,NaN,NaN,NaN,NaN,NaN,36061,NY,New York County,Manufacturing,2018
3,10001,42----,1079,662.0,198.0,109.0,69.0,23.0,15.0,3.0,NaN,NaN,36061,NY,New York County,Wholesale Trade,2018
4,10001,44----,654,387.0,127.0,62.0,38.0,18.0,8.0,11.0,NaN,NaN,36061,NY,New York County,Retail Trade,2018


In [9]:
allYearsTest = allYears
allYearsTest['year'] = allYearsTest['year'] -1

In [19]:
industries = list(np.unique(allYearsTest.loc[:100]['naics']))
industries = industries[1:]
print(list(map(lambda x: naics17[x], industries)))

['Construction', 'Manufacturing', 'Wholesale Trade', 'Retail Trade', 'Finance and Insurance', 'Real Estate and Rental and Leasing', 'Professional, Scientific, and Technical Services', 'Administrative and Support and Waste Management and Remediation Services', 'Health Care and Social Assistance', 'Accommodation and Food Services', 'Other Services (except Public Administration)']


In [12]:
import itertools
def createDifferences(df, f, col, n):
        df0 = df.loc[f]
        res0 = 100 * np.diff(df0[col], n)/df0[col][:-n]
        result = list(itertools.chain([np.nan] * n, res0))
        return result
    
def createTimeDif(df, county, column):
    listLoanChange = [createDifferences(df, f, column, 1) for f in county]
    return listLoanChange

In [13]:
import itertools
def createDifferences2(df, f, col, n):
        df0 = df.loc[f]
        res0 = np.diff(df0[col], n)
        result = list(itertools.chain([np.nan] * n, res0))
        return result
    
def createTimeDif2(df, county, column):
    listLoanChange = [createDifferences2(df, f, column, 1) for f in county]
    return listLoanChange

In [20]:
for industry in industries:
    temp = allYearsTest[allYearsTest['naics'] == industry]
    gSectors = temp.groupby(["FIPS","state","county","year"])["est", "n1_4","n5_9","n10_19","n20_49","n50_99","n100_249","n250_499","n500_999","n1000"].sum()
    counties = np.unique(list(map(lambda x: x[0], gSectors.index)))
    gSectors[['n1_4', 'n5_9', 'n10_19']] = gSectors[['n1_4', 'n5_9', 'n10_19']].fillna(0)
    gSectors["n1_19"] = gSectors["n1_4"] + gSectors["n5_9"] + gSectors["n10_19"]
    gSectors['n1_19_change'] = list(itertools.chain.from_iterable(createTimeDif2(gSectors, counties, "n1_19")))
    gSectors['est_change'] = list(itertools.chain.from_iterable(createTimeDif2(gSectors, counties, "est")))
    gSectors['naics'] = [industry]* gSectors.shape[0]
    gSectors['naics_desc'] = list(map(lambda x: naics17[x], gSectors['naics']))
    filename = "industry_data/" + industry+".csv"
    gSectors.to_csv(filename)
    print(industry)

23----
31----
42----
44----
52----
53----
54----
56----
62----
72----
81----


In [21]:
pd.Series(gSectors['n1_19'] - gSectors['n1_4'] - gSectors['n5_9'] - gSectors['n10_19']).value_counts()

0.0    61122
dtype: int64